In [3]:
!pip install openai weave wandb pandas numpy matplotlib seaborn python-docx nbformat
import os
import re
import weave
import wandb
import pandas as pd
import numpy as np
from openai import OpenAI
from typing import Dict, List, Any
import json

# 初始化 Weave 和 W&B
weave.init('oral-rewriting-1121')

# DeepSeek 客户端配置
client = OpenAI(
    api_key=os.environ.get('DEEPSEEK_API_KEY'),
    base_url="https://api.deepseek.com/v1"
)

POSITIVE_EXAMPLE="""
        原始转录片段：
            说话人2
            当时您就继续在数学系。
            说话人1
            第一那个时候还没计算机对吧？所以毕业就留在数学去了，而且我留下来因为去搞概率统计的。
            说话人2
            嗯嗯当时概率统计是您自己对这个方向有兴趣，还是说不知道？
            说话人1
            还应该过去过去在五六年应该好像搞了一个科学发展规划上，五六年国家制定这么一个规划，搞这个规划的话，当时好像当时我们反正确实也不太一样，我还听说好像他们到北京开会的时候说就但是对他是跟苏联关系也比较好，所以定规划的时候就苏联专家也来参加。
            说话人1
            当时他们好像认为在数学领域，我们听到了这个就是说好像就是说我们现在的对当时的中国的数学好像就有点陈旧的东西，好像。
            说话人1
            有一些好像国外很重要的数学分支没有得到好像应该没有。
            说话人2
            得到好像没有很好的解释。
            说话人1
            对，因为过去比如说那么过去一些老专家，他们因为都是也从外面回来，所以所以艺术团就是要对某些数学重要的学科要加强。这里面大概提到三样东西，一个就是计算机或者计算数学，那可以管这一套就跟计算机有关的一套、数学。另外一个就是说微用方程。还有一个就是说概率统计，就好像说三这三个东西，在当时国外至少或者说苏联专家或者制定规划的那些做大大专家们认为好像这个东西还是当时在数学应该是比较重要的，应该还是国内要
            说话人1
            还有这方面相对也比较弱，要加强这方面。
            说话人1
            所以在我们回来以后好像大概那个时候扩大数学系，也加强这方面国家都采取措施了，好像跟教研组也是也是在这个时候成立的，应该给了。
        优秀的口述史写法：
            “我毕业那会儿，还没有计算机呢，所以毕业就留在数学系了。我留下来搞概率统计，这个选择啊，跟当时国家的一个大规划有关系。
            大概是1956年吧，国家搞了一个科学发展规划。那时候我们和苏联关系好，制定规划的时候还有苏联专家来参加。我听说去北京开会的人回来说，专家们觉得我们中国的数学有点'陈旧'，国外一些重要的数学分支，我们这里要么没有，要么发展得不好。
            那些从国外回来的老专家也觉得是这样。所以规划里就定了，要重点加强几个数学学科：一个是计算数学（就是跟计算机有关的数学），一个是微分方程，还有一个是概率统计。这些领域当时在国外挺受重视的，但我们国内比较弱，所以要加紧发展。
            按照这个规划，我们数学系后来就扩大了，国家也给了支持，概率统计教研组就是那时候成立的。我留在数学系搞概率统计，也是跟着这个趋势走——既是国家需要，也是学科发展的好时机。"
        其优点：保留了老人朴实自然的语言风格，将口语整理为通顺的叙述，内容真实，并体现了时代背景。"""
        
# 改进的提示词模板定义
IMPROVED_PROMPT = """

请你扮演一位富有经验的传记作家和历史学者，专门从事口述史的整理与撰写。你的任务是根据一段老年人的采访转录文本，生成一篇真挚、真实、生动、结构清晰的老年人口述史。

首先，请基于以下指导原则，深度理解你将要去执行的“口述史生成”任务。
1. 任务定义：口述史是一种基于个人亲历的叙事文体。它不是简单的问答记录，而是将口语化的、可能零散的回忆，提炼、整合、重组为一条清晰的叙事线。它需要：
    （1）时间脉络：以时间顺序为骨架，勾勒出人生的主要阶段。
    （2）主题聚焦：突出{theme_focus}
    （3）情感与细节：保留受访者独特的语言风格、情感表达和关键的原话与细节，使其人物形象丰满。
    （4）叙事性：语言应平实流畅，富有故事性，读起来像一篇连贯的个人传记。
    （5）语言风格：保持老人朴实自然的语言风格，避免过度书面化。可以适当使用老人常用的口语词汇。

2. 需避免的事项（非常重要！）：
    （1）禁止虚构：绝不能添加或捏造任何采访中未提及的事实。
    （2）禁止罗列：避免以“他说……然后他说……”这种机械的方式拼接文本。
    （3）风格不当：避免使用过于书面化、学术化或官方报告式的语言。同时也要避免过于网络化的用语。
    （4）遗漏重点：不得忽略{key_events}的描述。

3. 正面示例:
{positive_example}

现在，请针对以下提供的采访转录文本，参考示例以及“任务定义”和“需避免的事项”中的所有要求，按照思维链步骤分析和改写口述内容，确保保留所有重要细节：

第一步：详细提取时间线和关键事件
识别所有时间线索和具体事件
建立完整的时间顺序
标记每个重要事件的具体细节

第二步：分析人物关系和互动
识别所有提到的人物及其关系
记录人物之间的具体互动和对话
分析人物情感变化

第三步：梳理事件逻辑和因果关系
提取所有主要和次要事件
分析事件之间的因果关系
保留事件的具体情境和背景

第四步：理解情感基调和细节
分析原始情感表达的细微差别
保持情感真实性，不美化负面情绪
保留情感表达的具体触发点

第五步：基于以上详细分析进行改写，确保：
保留原文的具体细节
字数不少于原文的20%
维持原文的个人化表达风格
在最终成文时，特别注意：
保持老人朴实自然的语言风格
适当使用口语词汇增强真实感
确保语言连贯、情感真挚、叙事生动

重要要求：
保持原文的所有重要细节和具体内容，不要过度概括
保留原文的情感和个人化表达
保持老人朴实自然的语言风格，可以适当保留老人常用的口语词汇和表达习惯，可以适当使用"那时候"、"那会儿"、"记得"等老人常用的口语词汇。
确保叙事连贯、情感真挚、生动有趣
字数不少于原文的20%
按照时间顺序和主题进行组织，但不要丢失具体事例

采访转录文本：
{oral_text}

【最终输出格式】

请将你的最终思考过程和成果，按以下清晰的结构呈现给我：

【步骤一：信息梳理结果】
- 时间线和关键事件：
- 人物关系和互动：
- 事件逻辑和因果关系：
- 情感基调和细节：

【步骤二：文章大纲】

【步骤三：生成的口述史正文】
（在这里输出完整的口述史文章）

"""

BASE_PROMPT_TEMPLATE= """
你是一个专业的口述历史整理者。请参考示例并按照思维链步骤分析和改写口述内容，确保保留所有重要细节：

示例改写： {positive_example}

第一步：详细提取时间线和关键事件
识别所有时间线索和具体事件
建立完整的时间顺序
标记每个重要事件的具体细节

第二步：分析人物关系和互动
识别所有提到的人物及其关系
记录人物之间的具体互动和对话
分析人物情感变化

第三步：梳理事件逻辑和因果关系
提取所有主要和次要事件
分析事件之间的因果关系
保留事件的具体情境和背景

第四步：理解情感基调和细节
分析原始情感表达的细微差别
保持情感真实性，不美化负面情绪
保留情感表达的具体触发点

第五步：基于以上详细分析进行改写，确保：
保留原文的具体细节
字数不少于原文的20%
维持原文的个人化表达风格

重要要求：
保持原文的所有重要细节和具体内容，不要过度概括
保留原文的情感和个人化表达
字数不少于原文的20%
按照时间顺序和主题进行组织，但不要丢失具体事例

现在请改写以下内容： {oral_text}
"""

class OralHistoryRewriter:
    def __init__(self):
        self.prompt_templates = {
            'base': BASE_PROMPT_TEMPLATE,
            'improved': IMPROVED_PROMPT,
        }
    
    @weave.op()
    def rewrite_text(self, method: str, oral_text: str, temperature: float = 0.3) -> str:
        """使用指定方法重写口述文本"""
        prompt_template = self.prompt_templates[method]
        if method == 'base':
            prompt = prompt_template.format(
                positive_example=POSITIVE_EXAMPLE,
                oral_text=oral_text
            )
        else:  # improved 方法
            prompt = prompt_template.format(
                oral_text=oral_text,
                theme_focus="求学经历、职业生涯、复旦计算机学院历史事件的个人见证",
                key_events="复旦计算机学院历史事件的个人见证",
                positive_example=POSITIVE_EXAMPLE
            )
        
        try:
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=[
                    {"role": "system", "content": "你是一位富有经验的传记作家和历史学者，专门从事口述史的整理与撰写。"},
                    {"role": "user", "content": prompt}
                ],
                temperature=temperature,
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"API调用错误: {str(e)}")
            return f"Error: {str(e)}"

class LongTextOralHistoryExperiment:
    def __init__(self):
        self.rewriter = OralHistoryRewriter()
    
    def read_docx_file(self, file_path: str) -> str:
        """读取docx文件内容"""
        try:
            from docx import Document
            doc = Document(file_path)
            full_text = []
            for paragraph in doc.paragraphs:
                if paragraph.text.strip():
                    full_text.append(paragraph.text)
            return '\n'.join(full_text)
        except ImportError:
            print("❌ 未安装python-docx库，无法读取docx文件")
            print("请运行: pip install python-docx")
            return ""
        except Exception as e:
            print(f"❌ 读取docx文件时出错: {str(e)}")
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    return f.read()
            except:
                return ""
    
    def clean_interview_text(self, text: str) -> str:
        """清理采访文本，移除时间戳和说话人标记"""
        # 移除时间戳格式 (00:00:00)
        text = re.sub(r'\(\d{2}:\d{2}:\d{2}\):', '', text)
        # 移除说话人标记
        text = re.sub(r'访谈人:', '', text)
        text = re.sub(r'Jenny:', '', text)
        # 合并多余的空行
        text = re.sub(r'\n\s*\n', '\n\n', text)
        return text.strip()
    
    def process_with_method(self, long_text: str, method: str, experiment_name: str) -> Dict:
        """使用特定方法处理整个长文本"""
        # 清理文本
        cleaned_text = self.clean_interview_text(long_text)
        print(f"  开始处理整个文本 (原始长度: {len(long_text)} 字符, 清理后: {len(cleaned_text)} 字符)")
        
        # 启动W&B
        wandb.init(
            project="oral-rewriting-1121",
            name=experiment_name,
            config={
                "method": method,
                "original_length": len(long_text),
                "cleaned_length": len(cleaned_text)
            },
            reinit=True
        )
        
        # 直接处理整个文本
        rewrite = self.rewriter.rewrite_text(method, cleaned_text)
        
        # 检查是否成功改写
        is_rewritten = (rewrite != cleaned_text and 
                       len(rewrite) > len(cleaned_text) * 0.5 and 
                       not rewrite.startswith("Error:"))
        
        # 创建结果记录
        result = {
            'original_text': long_text,
            'cleaned_text': cleaned_text,
            'rewritten_text': rewrite,
            'original_length': len(long_text),
            'cleaned_length': len(cleaned_text),
            'rewritten_length': len(rewrite),
            'compression_ratio': len(rewrite) / len(cleaned_text) if len(cleaned_text) > 0 else 1,
            'is_rewritten': is_rewritten
        }
        
        # 计算统计信息
        stats = {
            "original_length": len(long_text),
            "cleaned_length": len(cleaned_text),
            "rewritten_length": len(rewrite),
            "compression_ratio": len(rewrite) / len(cleaned_text) if len(cleaned_text) > 0 else 1,
            "is_rewritten": is_rewritten
        }
        
        # 记录到W&B
        wandb.log(stats)
        wandb.log({
            "original_text_sample": long_text[:800] + "..." if len(long_text) > 800 else long_text,
            "rewritten_text_sample": rewrite[:800] + "..." if len(rewrite) > 800 else rewrite
        })
        
        wandb.finish()
        
        return {
            "full_rewritten_text": rewrite,
            "result": result,
            "statistics": stats
        }
    
    def run_complete_experiment(self, long_text_path: str, experiment_name: str):
        """运行完整的长文本实验"""
        # 读取长文本
        if long_text_path.endswith('.docx'):
            long_text = self.read_docx_file(long_text_path)
        else:
            with open(long_text_path, 'r', encoding='utf-8') as f:
                long_text = f.read()
        
        if not long_text:
            print("❌ 无法读取文件内容")
            return {}
        
        print(f"📖 读取文本，总长度: {len(long_text)} 字符")
        
        # 为每种方法运行实验
        all_results = {}
        
        for method in ['base', 'improved']:
            print(f"\n🔬 开始方法: {method}")
            
            method_experiment_name = f"{experiment_name}_{method}_1121"
            result = self.process_with_method(long_text, method, method_experiment_name)
            all_results[method] = result

        return all_results

def run_improved_experiment():
    """运行改进后的实验"""
    print("🎯 开始改进版老年人口述史AI改写实验...")
    
    # 初始化实验
    experiment = LongTextOralHistoryExperiment()
    
    # 运行实验
    long_text_path = "访谈语音转文字稿.docx"
    experiment_name = "老年人采访改进实验"
    
    results = experiment.run_complete_experiment(long_text_path, experiment_name)
    
    if results:  # 先判断实验结果是否有效
        # 定义两种方法对应的文件名（严格按你要求的格式）
        method_to_filename = {
            "base": "rewritten_base_老年人采访实验_1121_v5.txt",
            "improved": "rewritten_improved_老年人采访实验_1121_v5.txt"
        }
        
        for method in ["base", "improved"]:
            if method in results:
                rewritten_text = results[method]["full_rewritten_text"]
                if rewritten_text.startswith("Error:"):
                    print(f"\n❌ {method} 方法生成失败，不保存文件：{rewritten_text}")
                    continue
                target_filename = method_to_filename[method]
                try:
                    with open(target_filename, "w", encoding="utf-8") as f:
                        f.write(rewritten_text)
                    print(f"\n✅ {method} 方法结果已保存为：{target_filename}")
                    print(f"   （文件路径：与当前运行的代码文件在同一文件夹下）")
                except Exception as e:
                    print(f"\n❌ 保存 {method} 方法结果失败：{str(e)}")
    
    return results


if __name__ == "__main__":
    run_improved_experiment()
    
    

🎯 开始改进版老年人口述史AI改写实验...
📖 读取文本，总长度: 8175 字符

🔬 开始方法: base
  开始处理整个文本 (原始长度: 8175 字符, 清理后: 8175 字符)



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


wandb: Initializing weave.


c:\Users\yuanzhouyan\AppData\Local\Programs\Python\Python313\Lib\site-packages\rich\live.py:256: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

weave: wandb version 0.23.0 is available!  To upgrade, please run:
weave:  $ pip install wandb --upgrade
weave: weave version 0.52.17 is available!  To upgrade, please run:
weave:  $ pip install weave --upgrade
weave: Logged in as Weights & Biases user: yuan30jancis.
weave: View Weave data at https://wandb.ai/yuan30jancis-fudan-university-school-of-management/oral-rewriting-1121/weave
weave: 🍩 https://wandb.ai/yuan30jancis-fudan-university-school-of-management/oral-rewriting-1121/r/call/019aa71a-e6a6-7cfe-89c6-525d0ad6cca2


...

c:\Users\yuanzhouyan\AppData\Local\Programs\Python\Python313\Lib\site-packages\rich\live.py:256: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

weave: Flushing 2 pending tasks...


weave: wandb version 0.23.0 is available!  To upgrade, please run:
weave:  $ pip install wandb --upgrade
weave: weave version 0.52.17 is available!  To upgrade, please run:
weave:  $ pip install weave --upgrade
weave: Logged in as Weights & Biases user: yuan30jancis.
weave: View Weave data at https://wandb.ai/yuan30jancis-fudan-university-school-of-management/oral-rewriting-1121/weave
weave: 🍩 https://wandb.ai/yuan30jancis-fudan-university-school-of-management/oral-rewriting-1121/r/call/019aa71c-3fd5-75a8-af16-110935c0c4a1


cleaned_length,▁
compression_ratio,▁
original_length,▁
rewritten_length,▁
cleaned_length,8175
compression_ratio,0.25199
is_rewritten,False
original_length,8175
original_text_sample,2：那我们就开始了，我们看到网上的公开资...
rewritten_length,2060
rewritten_text_sample,【步骤一：信息梳理结果】- 时间线和...



✅ base 方法结果已保存为：rewritten_base_老年人采访实验_1121_v5.txt
   （文件路径：与当前运行的代码文件在同一文件夹下）

✅ improved 方法结果已保存为：rewritten_improved_老年人采访实验_1121_v5.txt
   （文件路径：与当前运行的代码文件在同一文件夹下）
